# EMToolKit Dashboard Example
## Multi-Instrument Method

### Import Statements

In [ ]:
# %pip install astropy scipy ipympl ndcube numba xrtpy sunpy
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

import importlib
import numpy as np
from sys import path
import astropy.units as u
import matplotlib.pyplot as plt

# Enable interactive plotting with widgets
%matplotlib widget

# Set matplotlib parameters for better visualization
plt.rcParams.update({'image.origin': 'lower'})
np.set_printoptions(linewidth=128)

# Define base path and append EMToolKit path
base_path = "../"
emtk_path = base_path + 'code/'
path.append(emtk_path)

# Import EMToolKit and its visualization module
import EMToolKit.EMToolKit as emtk
import EMToolKit
import EMToolKit.visualization

# Reload EMToolKit modules to ensure the latest version is used
importlib.reload(EMToolKit)
importlib.reload(EMToolKit.visualization)

# Initialize an empty EM collection
em_collection = 0

In [ ]:
# These flags tell the program not to recalculate the data if it has already been calculated
redownload = False
recalc_sparse = False
recalc_simple = False

### Download the Imagery

In [ ]:
from EMToolKit.instruments.aia import download_sdo_data
from EMToolKit.instruments.xrt import download_xrt_data

date='2010/07/25 17:58:44'

# Download the data
aia_paths, sdo_data_dir = download_sdo_data(base_path, date, redownload=redownload)
xrt_paths, xrt_data_dir = download_xrt_data(base_path, date, redownload=redownload)
paths = aia_paths + xrt_paths




In [ ]:
import xrtpy
import matplotlib.pyplot as plt
import numpy as np

# Define filter and date-time for the temperature response
filter = "C-poly"
date_time = "2007-09-22T21:59:59"

# Create a TemperatureResponseFundamental object
Temperature_Response_Fundamental = xrtpy.response.TemperatureResponseFundamental(
    filter, date_time, abundance_model="Photospheric"
)

# Get the temperature response and CHIANTI temperature
temperature_response = Temperature_Response_Fundamental.temperature_response()
CHIANTI_temperature = Temperature_Response_Fundamental.CHIANTI_temperature
log_CHIANTI_temperature = np.log10(CHIANTI_temperature.value)

# Print the log of the CHIANTI temperature and the temperature response
# print("log_CHIANTI_temperature:\n", log_CHIANTI_temperature)
# print("Temperature Response:\n", temperature_response)

### Load and Crop the Imagery

In [ ]:
from EMToolKit.instruments.aia import load_from_paths, aia_wrapper
from EMToolKit.instruments.xrt import xrt_wrapper

xl, yl, = 240*u.arcsec, -525*u.arcsec # Left edge of FOV in arcseconds (relative to disk center)
                                      # Right edge of FOV in arcseconds (relative to disk center)
xlen, ylen  = 270*u.arcsec, 240*u.arcsec # X size of FOV in arcseconds # Y size of FOV in arcseconds


# Load the AIA Images ###
aiamaps = load_from_paths(aia_paths,xl=xl,yl=yl,dx=xlen,dy=ylen)

temperature_array = np.arange(5.5, 7.5, 0.025)

[maps,errs,trlogts,tresps] = aia_wrapper(aiamaps, temperature_array=temperature_array)

# maps[0].data[30:50, :] = -40
# maps[0].data[55:75, :] = -60
# maps[0].peek()

# Load the XRT Image ###
    # We're loading these separately so that we can apply an offset to the XRT data -- there appears
    # to be a significant misalignment based on manual inspection and testing:
xrt_offsets = [-5*u.arcsec, -20*u.arcsec]
xrtmap = load_from_paths([[paths[0]],xrt_paths[0]],xl=xl+xrt_offsets[0],yl=yl+xrt_offsets[1],dx=xlen,dy=ylen)[1]
xrtmap.meta['crval1'] -= xrt_offsets[0].value
xrtmap.meta['crval2'] -= xrt_offsets[1].value
[xrtmaps,xrterrs,xrt_trlogts,xrt_tresps] = xrt_wrapper([xrtmap], temperature_array=temperature_array)
# Append the XRT data to the AIA data
maps.append(xrtmaps[0])
errs.append(xrterrs[0])
trlogts.append(xrt_trlogts[0])
tresps.append(xrt_tresps[0])


In [ ]:
print(len(trlogts[0]))
print(len(trlogts[-1]))

In [ ]:
# Normalizing the response functions makes them much easier for the solver to handle.
datmax_nominal = 1.0e4 # The nominal maximum of the data
overall_norm = datmax_nominal/np.max(np.array(tresps))
norms = np.max(np.array(tresps),axis=0)/np.max(np.array(tresps))/overall_norm

In [ ]:
# Make data sequence from AIA & XRT data:
datasequence = emtk.em_data(maps,errs,trlogts,tresps)

# Create em_collection with data sequence:
em_collection = emtk.em_collection(datasequence)
em_collection_HQ = emtk.em_collection(datasequence)

In [ ]:
# datasequence[0].meta

In [ ]:
# Show some of the AIA channels and their temperature responses for illustration:
plt_emmax = 5.0e28  # Maximum emission value for plotting
gfac = 1.0 / 2.2  # Gamma correction factor for visualization

fig, axs = plt.subplots(2, 3)  # Create a figure with a 2x3 grid of subplots

# Plot AIA 193 Angstrom channel
axs[0, 0].imshow(np.clip(maps[1].data / np.max(tresps[1]), 0, plt_emmax) ** gfac, cmap=plt.get_cmap('gray'))
axs[0, 0].set(title='AIA 193 Angstrom')
axs[1, 0].plot(trlogts[1], tresps[1] / 1.0e-27)
axs[1, 0].set(title='AIA 193 TResp', xlabel='Temperature (dB Kelvin)', ylabel='Response (DN/s/(10$^9$ cm$^{-3})^2$/Mm)')

# Plot AIA 335 Angstrom channel
axs[0, 1].imshow(np.clip(maps[2].data / np.max(tresps[2]), 0, plt_emmax) ** gfac, cmap=plt.get_cmap('gray'))
axs[0, 1].set(title='AIA 335 Angstrom')
axs[1, 1].plot(trlogts[2], tresps[2] / 1.0e-27)
axs[1, 1].set(title='AIA 335 TResp', xlabel='Temperature (dB Kelvin)')

# Plot AIA 94 Angstrom channel
axs[0, 2].imshow(np.clip(maps[5].data / np.max(tresps[5]), 0, plt_emmax) ** gfac, cmap=plt.get_cmap('gray'))
axs[0, 2].set(title='AIA 94 Angstrom')
axs[1, 2].plot(trlogts[5], tresps[5] / 1.0e-27)
axs[1, 2].set(title='AIA 94 TResp', xlabel='Temperature (dB Kelvin)')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

### Compute the Differential Emission Measure

In [ ]:
from EMToolKit.algorithms.multi_inst_dem_wrapper import multi_inst_simple_dem_wrapper
importlib.reload(EMToolKit.algorithms.multi_inst_dem_wrapper)
importlib.reload(EMToolKit)
from EMToolKit.algorithms.multi_inst_dem_wrapper import multi_inst_simple_dem_wrapper

# Make DEM sequence from the DEM and add it to the collection
multi_em_demsequence, multi_out = multi_inst_simple_dem_wrapper(
    datasequence,
    wrapargs={'norms': norms, 'overall_norm': overall_norm},
    recalc_simple=recalc_simple
)
# multi_coeffs, multi_logts, multi_bases, multi_coords, multi_algorithm = multi_out
em_collection.add_model(multi_em_demsequence)

## Run the Dashboard

In [ ]:
# from __future__ import print_function


# Get the most recent version from the tool kit:
from EMToolKit.visualization.dashboard import dashboard_object
importlib.reload(EMToolKit.visualization.dashboard)
from EMToolKit.visualization.dashboard import dashboard_object

# Suppress specific SunpyMetadataWarning
import warnings
from sunpy.util.exceptions import SunpyMetadataWarning
warnings.filterwarnings("ignore", category=SunpyMetadataWarning)

# Create the dashboard:
dash = dashboard_object(em_collection)
dash.display()




In [ ]:

# multi_em_demsequence = emtk.dem_model(*multi_out)
# Unpack the output


In [ ]:
# # print(type((collect)))
# # collect_dict = dict(collect)
# # print(collect_dict.keys())
# # print(collect_dict["data"])


# import asdf
# from ndcube import NDCollection

# # Your NDCollection
# my_ndcollection = collect

# # Create a new ASDF file
# with asdf.AsdfFile() as af:
#     # Iterate over items in the NDCollection and add them to the ASDF file
#     # print(my_ndcollection['models'])

#     af['data'] = []
#     af['meta_keys'] = []
#     af['meta_values'] = []
#     af['methods'] = []
#     for img in my_ndcollection['data']:
#         af['data'].append(img.data)
#         img_meta = copy.copy(img.meta)
#         # schema = img_meta["schema"]
#         # del img_meta["schema"]
#         # print (img_meta)
#         af['meta_keys'].append(img_meta.keys())
#         af['meta_values'].append(img_meta.values())

#     for mthd in my_ndcollection['models']:
#         af['methods'].append(mthd)
#         # af['method_meta'].append(([key, value] for key, value in mthd.meta.items()))

#     for key, ndcube in my_ndcollection.items():
#         # Here, 'key' can be used to uniquely identify each NDCube in the collection
#         # print (dir(ndcube))
#         # af[key] = {
#         #     'data': ,
#         #     # 'meta': ndcube.meta
#         # }
#         pass
#         # print(key)
#         # print((my_ndcollection[key]))
#     # Write the ASDF file to disk
#     af.write_to(f'{data_path}/my_ndcollection.fits')


# # print(collect_dict['data'][0].wcs)

In [ ]:
# multi_synthmaps = [print(map.data.shape) for map in em_collection.data()[:-1]]
# multi_synthmaps = [em_collection.synthesize_map(map) for map in em_collection.data()[:-1]]
# multi_synthdata = [map for map in multi_synthmaps]

In [ ]:
# def calc_resids(synthdata, em_collection):# Calculate the residuals and Chi squared:
# 	ndata = len(synthdata)
# 	resids = []
# 	datasequence = em_collection.data()
# 	chi2 = 0
# 	[nx,ny] = datasequence[0].data.shape
# 	for seq in datasequence: [nx,ny] = [np.min([seq.data.shape[0],nx]),np.min([seq.data.shape[1],ny])]
# 	for i in range(0,ndata):
# 		resids.append(((synthdata[i].data-datasequence[i].data)/datasequence[i].uncertainty.array)**2)
# 		chi2 += np.mean(resids[i])/ndata
# 	return resids, chi2

# # #spars_resids, spars_chi2 = calc_resids(spars_synthdata,em_collection)
# # #simpl_resids, simpl_chi2 = calc_resids(simpl_synthdata,em_collection)
# multi_resids, multi_chi2 = calc_resids(multi_synthdata, em_collection)
# print("Multi-instrument Chi squared = ",multi_chi2)
# # #print('simple_reg_dem Chi squared = ',simpl_chi2, 'spars_em Chi squared = ',spars_chi2)

In [ ]:
# # Plot Residuals:
# fig = plt.figure(figsize=[20,12])
# plt.suptitle('Residuals for '+multi_algorithm)
# for i in range(0,7):
#     ax1 = fig.add_subplot(2,4,i+1)
#     ax1.imshow(multi_resids[i],vmin=0,vmax=5)
#     ax1.set(title=em_collection.data()[i].meta['channel'])